In [2]:
import os
import sys

# allow import from parent directory (we are not in a module)
sys.path.insert(1, os.path.join(sys.path[0], '..'))

import whisper

In [4]:
model = whisper.load_model("base")

# load audio and pad/trim it to fit 30 seconds
audio = whisper.load_audio("../tests/jfk.flac")
audio = whisper.pad_or_trim(audio)

# make log-Mel spectrogram and move to the same device as the model
mel = whisper.log_mel_spectrogram(audio).to(model.device)

# detect the spoken language
_, probs = model.detect_language(mel)
print(f"Detected language: {max(probs, key=probs.get)}")

Detected language: en


In [9]:
# decode the audio
options = whisper.DecodingOptions(
    language='en',
    fp16=False,  # needed for m1 macs?
)
result = whisper.decode(model, mel, options)

torch.Size([1500, 512])

In [13]:
result.text

'And so my fellow Americans, ask not what your country can do for you, ask what you can do for your country.'